# 위쪽 부분은 TEST, 아래쪽이 함수화한 부분입니다!

# SEAT 처리 : 완료

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings

plt.rc('font', family = 'Malgun Gothic')
filterwarnings('ignore')

# 전체 데이터 출력 및 확인 옵션 설정
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.options.display.max_info_columns

100

In [3]:
# place에 따라 자리가 다를 것으로 생각하고, 나눠서 작업하기
df = pd.read_csv("dataset/2023bigcontest_advanced_sac.csv")
df['place'].unique()

array(['콘서트홀', '리사이틀홀', 'IBK챔버홀'], dtype=object)

In [4]:
df_concert = df.query("`place`=='콘서트홀'")
df_chamber = df.query("`place`=='IBK챔버홀'")
df_recital = df.query("`place`=='리사이틀홀'")

In [5]:
df_concert.seat.sample(10)
# 3층 N블록1열 9 | 합창석 F블록4열 7 | 2층 BOX1 4 와 같이 split으로 3묶음으로 나뉘며, '블록'을 기준으로 나누어 담으면 될듯함. BOX는 '블록'으로 취급하기로 함.

357986      1층 D블록10열 8
24510        3층 A블록5열 5
1477862      2층 E블록4열 2
140283      2층 A블록7열 16
1715212      2층 E블록5열 5
1336245    1층 D블록21열 13
35284        2층 C블록6열 7
1790989     1층 E블록18열 4
822405       1층 E블록6열 5
514207      1층 B블록22열 8
Name: seat, dtype: object

In [12]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_concert1 = df_concert.copy() #실험용

# 1.seat_floor 생성
df_concert1['seat_floor'] = df_concert1['seat'].str.split(" ").str[0]
df_concert1['seat_floor'] = df_concert1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제
df_concert1['seat_floor'] = df_concert1['seat_floor'].replace("합창석",4,regex=True) # 3층까지만 있으므로, 합창석은 4로 대체
df_concert1['seat_floor'].astype(int)

# 2. seat_block, seat_col 생성 
# 2-1. BOX1과 같이 "블록"으로 split 불가한 경우 대비 : seat_block에 넣어두고, 이 경우 seat_col은 nan이 되므로 -1 부여
df_concert1['seat_block'] = df_concert1['seat'].str.split(" ").str[1]
df_concert1['seat_col'] = -1

# 2-2. box df, block df 두 가지로 구분
df_concert_box = df_concert1.query('not seat_block.str.contains("블록")')
df_concert_block = df_concert1.query('seat_block.str.contains("블록")')

# 2-3. block df에서 "블록"기준으로 구분하여 담고, "열"글자 삭제
df_concert_block[['seat_block', 'seat_col']] = df_concert_block['seat_block'].str.split("블록", expand=True)
df_concert_block['seat_col'] = df_concert_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

df_concert2 = pd.concat([df_concert_box, df_concert_block])
df_concert2['seat_col'].astype(int)    
df_concert2

# 3. seat_num 생성
df_concert2['seat_num'] = df_concert2['seat'].str.split(" ").str[-1]
df_concert2['seat_num'].astype(int)

df_concert2

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,seat_floor,seat_block,seat_col,seat_num
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,20220204,2000,3층 BOX9 10,10000,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y,3,BOX9,-1,10
11,50.0,F,무료,NaN,NaN,NaN,NaN,NaN,20191005,1417,20191201,1700,3층 BOX9 5,24000,0,K-lang 멤버십 회원(1인2매)20%,315,NaN,20191005.0,오페라,콘서트홀,120,15,Y,3,BOX9,-1,5
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220624,1618,20220629,1930,3층 BOX12 3,20000,2,일반,2344,20220430.0,20220430.0,교향곡,콘서트홀,110,15,N,3,BOX12,-1,3
18,70.0,M,골드,무료,NaN,NaN,NaN,NaN,20191220,1106,20200211,1930,2층 BOX2 2,42000,0,골드회원 할인30%,76,20191220.0,20191223.0,클래식,콘서트홀,120,15,Y,2,BOX2,-1,2
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20230507,2130,20230701,1700,3층 BOX11 5,40000,0,일반,2822,20230505.0,20230507.0,교향곡,콘서트홀,120,15,N,3,BOX11,-1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920853,60.0,F,무료,그린,NaN,NaN,NaN,NaN,20210618,1509,20210704,1500,1층 A블록2열 2,90000,2,일반,887,20210618.0,20210620.0,교향곡,콘서트홀,110,20,Y,1,A,2,2
1920855,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20230522,1729,20230613,1700,3층 A블록6열 4,10000,0,일반,2825,20230505.0,20230507.0,교향곡,콘서트홀,120,15,Y,3,A,6,4
1920856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20201009,1652,20201020,1930,1층 D블록16열 12,0,0,초대권,77,NaN,20200920.0,클래식,콘서트홀,120,15,N,1,D,16,12
1920858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20200726,1655,20200818,1930,1층 D블록20열 8,0,0,기획사판매,850,20200719.0,20200720.0,합창,콘서트홀,110,15,N,1,D,20,8


In [14]:
df_concert2.isnull().sum() #null 값 없음 확인 완료!

age                   612451
gender                612101
membership_type_1     612101
membership_type_2     707231
membership_type_3     956500
membership_type_4    1071808
membership_type_5    1095858
membership_type_6    1096968
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
seat                       0
price                      0
ticket_cancel              0
discount_type              0
performance_code           0
pre_open_date         395645
open_date                  0
genre                  25073
place                      0
running_time               0
intermission               0
member_yn                  0
seat_floor                 0
seat_block                 0
seat_col                   0
seat_num                   0
dtype: int64

In [15]:
df_chamber.seat.sample(10)
# 1층 B블록 1열 1 | 2층 BOX2 3 과 같음. 공백 기준 split으로 나눠담기.

830316      1층 B블록 3열 12
1896373    1층 B블록 11열 12
350746       1층 A블록 7열 3
28221       1층 A블록 13열 2
1915495      2층 A블록 3열 4
1471034      2층 B블록 2열 7
1500853      1층 C블록 1열 1
1278102    1층 B블록 14열 16
411769      1층 B블록 17열 7
531993        2층 BOX2 10
Name: seat, dtype: object

In [20]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_chamber1 = df_chamber.copy() #실험용

# 1.seat_floor 생성
df_chamber1['seat_floor'] = df_chamber1['seat'].str.split(" ").str[0]
df_chamber1['seat_floor'] = df_chamber1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음
df_chamber1['seat_floor'].astype(int)

# 2. seat_block 생성 
df_chamber1['seat_block'] = df_chamber1['seat'].str.split(" ").str[1]

# 3. seat_col 생성 :  box df, block df 두 가지로 구분
df_chamber_box = df_chamber1.query('not seat_block.str.contains("블록")')

# 3-1. BOX석이 seat_block으로 입력될 때 seat_col은 nan이 되므로 -1 부여
df_chamber_box['seat_col'] = -1

# 3-2. 블록이 있으면 구분되어 있으므로 데이터 부여
df_chamber_block = df_chamber1.query('seat_block.str.contains("블록")')

# 3-3. seat 데이터를 이용해 공백 기준으로 구분하여 담고, "블록"&"열"글자 삭제
df_chamber_block['seat_col'] = df_chamber_block['seat'].str.split(" ").str[2]

df_chamber_block['seat_block'] = df_chamber_block['seat_block'].replace("블록$","",regex=True) # 마지막 글자가 '블록'이면 삭제
df_chamber_block['seat_col'] = df_chamber_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

df_chamber2 = pd.concat([df_chamber_box, df_chamber_block])
df_chamber2['seat_col'].astype(int)    

# 3. seat_num 생성
df_chamber2['seat_num'] = df_chamber2['seat'].str.split(" ").str[-1]
df_chamber2['seat_num'].astype(int)

df_chamber2

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,seat_floor,seat_block,seat_col,seat_num
136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20230213,2010,20230219,1400,2층 BOX2 8,0,0,초대권,364,NaN,20230114.0,실내악,IBK챔버홀,90,0,N,2,BOX2,-1,8
312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20221205,2052,20221210,1400,2층 BOX1 10,0,0,초대권,1291,NaN,20221111.0,실내악,IBK챔버홀,90,15,N,2,BOX1,-1,10
330,30.0,F,골드,무료,싹틔우미,그린,NaN,NaN,20230317,1308,20230603,2000,2층 BOX2 5,21000,2,골드회원 할인30%,2441,20230317.0,20230318.0,실내악,IBK챔버홀,95,15,Y,2,BOX2,-1,5
1230,50.0,F,블루,골드,무료,NaN,NaN,NaN,20220927,1014,20221213,1930,2층 BOX1 6,28000,2,블루회원 할인5%,1665,20220729.0,20220730.0,독주,IBK챔버홀,95,15,Y,2,BOX1,-1,6
1351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190326,1300,20190330,2000,2층 BOX1 10,0,0,초대권,2602,NaN,20190225.0,실내악,IBK챔버홀,110,15,N,2,BOX1,-1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920860,60.0,M,골드,무료,그린,NaN,NaN,NaN,20190222,1409,20190508,2000,1층 B블록 2열 9,15000,2,블라인드티켓 할인(1인4매)50%,49,20190222.0,20190223.0,클래식,IBK챔버홀,100,15,Y,1,B,2,9
1920861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220701,1543,20220713,1930,1층 B블록 13열 2,0,0,기획사판매,1536,20220627.0,20220627.0,실내악,IBK챔버홀,120,15,N,1,B,13,2
1920863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190614,920,20190706,1400,1층 A블록 14열 6,0,0,초대권,1265,20190528.0,20190531.0,실내악,IBK챔버홀,100,15,N,1,A,14,6
1920866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211123,1617,20211127,2000,1층 B블록 9열 5,0,0,초대권,1920,20211029.0,20211029.0,독주,IBK챔버홀,90,15,N,1,B,9,5


In [21]:
df_chamber2.isnull().sum()

age                  374397
gender               374313
membership_type_1    374313
membership_type_2    400281
membership_type_3    450118
membership_type_4    472903
membership_type_5    477253
membership_type_6    477410
tran_date                 0
tran_time                 0
play_date                 0
play_st_time              0
seat                      0
price                     0
ticket_cancel             0
discount_type             0
performance_code          0
pre_open_date        251580
open_date                 0
genre                   769
place                     0
running_time              0
intermission              0
member_yn                 0
seat_floor                0
seat_block                0
seat_col                  0
seat_num                  0
dtype: int64

In [78]:
df_recital.seat.sample(10)
#  1층 6열 2 |  1층 BOX1 2열 23 | 1층 우측 1열 7 | 2층 좌측 1열 9 와 같음.

1819125          1층 9열 1
712144          1층 12열 9
941151          1층 5열 13
385004          1층 7열 15
1807432    2층 BOX3 1열 17
1444448          1층 3열 4
718369          1층 2열 14
593094           1층 9열 5
523954         1층 10열 10
1037999       2층 좌측 1열 9
Name: seat, dtype: object

In [26]:
# 'seat_floor', 'seat_block', 'seat_col', 'seat_num' 생성
df_recital1 = df_recital.copy() #실험용

# 1.seat_floor 생성
df_recital1['seat_floor'] = df_recital1['seat'].str.split(" ").str[0]
df_recital1['seat_floor'] = df_recital1['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음
df_recital1['seat_floor'].astype(int)

# 2. seat_block 생성 : block, noblock으로 구분 : split list의 길이가 3이면 nan 값이므로 -1 부여
df_recital_noblock = df_recital1[df_recital1['seat'].str.split(" ").str.len() < 4]
df_recital_noblock['seat_block'] = -1

df_recital_block = df_recital1[df_recital1['seat'].str.split(" ").str.len() == 4]
df_recital_block['seat_block'] = df_recital_block['seat'].str.split(" ").str[1]

df_recital2 = pd.concat([df_recital_noblock, df_recital_block])

# 3. seat_col 생성 : 뒤에서 두 번째 값으로 부여
df_recital2['seat_col'] = df_recital2['seat'].str.split(" ").str[-2]
df_recital2['seat_col'] = df_recital2['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제
df_recital2['seat_col'].astype(int)    

# 3. seat_num 생성
df_recital2['seat_num'] = df_recital2['seat'].str.split(" ").str[-1]
df_recital2['seat_num'].astype(int)

df_recital2

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,seat,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,seat_floor,seat_block,seat_col,seat_num
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211003,1550,20211009,2000,1층 7열 5,0,0,초대권,1120,NaN,20210823.0,독주,리사이틀홀,90,15,N,1,-1,7,5
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190630,1132,20190731,2000,1층 2열 3,0,0,초대권,2798,NaN,20190701.0,독주,리사이틀홀,90,15,N,1,-1,2,3
14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20230605,1104,20230610,2000,2층 2열 5,0,0,초대권,2750,NaN,20230424.0,독주,리사이틀홀,90,15,N,2,-1,2,5
23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20221128,1346,20221204,1400,2층 3열 9,0,0,초대권,583,20221029.0,20221029.0,독주,리사이틀홀,90,15,N,2,-1,3,9
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20221216,1826,20221220,1930,1층 4열 1,0,0,초대권,1132,20221105.0,20221105.0,독주,리사이틀홀,90,15,N,1,-1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191125,1135,20191130,1400,2층 BOX4 1열 12,0,0,초대권,2056,NaN,20191102.0,독주,리사이틀홀,90,15,N,2,BOX4,1,12
1920778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20191024,1403,20191204,2000,2층 BOX4 1열 12,0,0,초대권,2405,NaN,20191019.0,독주,리사이틀홀,100,15,N,2,BOX4,1,12
1920786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20210329,1751,20210427,1930,2층 BOX3 1열 9,0,0,초대권,1075,20210327.0,20210327.0,독주,리사이틀홀,90,15,N,2,BOX3,1,9
1920822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20230226,1737,20230404,1930,1층 BOX1 1열 7,0,0,기획사,16,NaN,20230224.0,실내악,리사이틀홀,100,15,N,1,BOX1,1,7


In [27]:
df_recital2.isnull().sum()

age                  313335
gender               313300
membership_type_1    313300
membership_type_2    325703
membership_type_3    342391
membership_type_4    345964
membership_type_5    346475
membership_type_6    346490
tran_date                 0
tran_time                 0
play_date                 0
play_st_time              0
seat                      0
price                     0
ticket_cancel             0
discount_type             0
performance_code          0
pre_open_date        217160
open_date               340
genre                     0
place                     0
running_time              0
intermission              0
member_yn                 0
seat_floor                0
seat_block                0
seat_col                  0
seat_num                  0
dtype: int64

# 함수화

In [29]:
def separate_seat(df):
    # 0. place별로 df 구분
    df_concert = df.query("`place`=='콘서트홀'")
    df_chamber = df.query("`place`=='IBK챔버홀'")
    df_recital = df.query("`place`=='리사이틀홀'")


    # ===concert 홀 처리===
    # 1.seat_floor 생성
    df_concert['seat_floor'] = df_concert['seat'].str.split(" ").str[0]
    df_concert['seat_floor'] = df_concert['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제
    df_concert['seat_floor'] = df_concert['seat_floor'].replace("합창석",4,regex=True) # 3층까지만 있으므로, 합창석은 4로 대체
    df_concert['seat_floor'].astype(int)

    # 2. seat_block, seat_col 생성 
    # 2-1. BOX1과 같이 "블록"으로 split 불가한 경우 대비 : seat_block에 넣어두고, 이 경우 seat_col은 nan이 되므로 -1 부여
    df_concert['seat_block'] = df_concert['seat'].str.split(" ").str[1]
    df_concert['seat_col'] = -1

    # 2-2. box df, block df 두 가지로 구분
    df_concert_box = df_concert.query('not seat_block.str.contains("블록")')
    df_concert_block = df_concert.query('seat_block.str.contains("블록")')

    # 2-3. block df에서 "블록"기준으로 구분하여 담고, "열"글자 삭제
    df_concert_block[['seat_block', 'seat_col']] = df_concert_block['seat_block'].str.split("블록", expand=True)
    df_concert_block['seat_col'] = df_concert_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    df_concert_new = pd.concat([df_concert_box, df_concert_block])
    df_concert_new['seat_col'].astype(int)    

    # 3. seat_num 생성
    df_concert_new['seat_num'] = df_concert_new['seat'].str.split(" ").str[-1]
    df_concert_new['seat_num'].astype(int)


    # ===chamber 홀 처리===
    # 1.seat_floor 생성
    df_chamber['seat_floor'] = df_chamber['seat'].str.split(" ").str[0]
    df_chamber['seat_floor'] = df_chamber['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음
    df_chamber['seat_floor'].astype(int)

    # 2. seat_block 생성 
    df_chamber['seat_block'] = df_chamber['seat'].str.split(" ").str[1]

    # 3. seat_col 생성 :  box df, block df 두 가지로 구분
    df_chamber_box = df_chamber.query('not seat_block.str.contains("블록")')
    # 3-1. BOX석이 seat_block으로 입력될 때 seat_col은 nan이 되므로 -1 부여
    df_chamber_box['seat_col'] = -1

    # 3-2. 블록이 있으면 구분되어 있으므로 데이터 부여
    df_chamber_block = df_chamber.query('seat_block.str.contains("블록")')

    # 3-3. seat 데이터를 이용해 공백 기준으로 구분하여 담고, "블록"&"열"글자 삭제
    df_chamber_block['seat_col'] = df_chamber_block['seat'].str.split(" ").str[2]
    df_chamber_block['seat_block'] = df_chamber_block['seat_block'].replace("블록$","",regex=True) # 마지막 글자가 '블록'이면 삭제
    df_chamber_block['seat_col'] = df_chamber_block['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제

    df_chamber_new = pd.concat([df_chamber_box, df_chamber_block])
    df_chamber_new['seat_col'].astype(int)    

    # 4. seat_num 생성
    df_chamber_new['seat_num'] = df_chamber_new['seat'].str.split(" ").str[-1]
    df_chamber_new['seat_num'].astype(int)

    # ===recital 홀 처리===
    # 1.seat_floor 생성
    df_recital['seat_floor'] = df_recital['seat'].str.split(" ").str[0]
    df_recital['seat_floor'] = df_recital['seat_floor'].replace("층$","",regex=True) # 마지막 글자가 '층'이면 삭제. 1층과 2층만 있음
    df_recital['seat_floor'].astype(int)

    # 2. seat_block 생성 : block, noblock으로 구분 : split list의 길이가 3이면 nan 값이므로 -1 부여
    df_recital_noblock = df_recital[df_recital['seat'].str.split(" ").str.len() < 4]
    df_recital_noblock['seat_block'] = -1

    df_recital_block = df_recital[df_recital['seat'].str.split(" ").str.len() == 4]
    df_recital_block['seat_block'] = df_recital_block['seat'].str.split(" ").str[1]

    df_recital_new = pd.concat([df_recital_noblock, df_recital_block])

    # 3. seat_col 생성 : 뒤에서 두 번째 값으로 부여
    df_recital_new['seat_col'] = df_recital_new['seat'].str.split(" ").str[-2]
    df_recital_new['seat_col'] = df_recital_new['seat_col'].replace("열$","",regex=True) # 마지막 글자가 '열'이면 삭제
    df_recital_new['seat_col'].astype(int)    

    # 3. seat_num 생성
    df_recital_new['seat_num'] = df_recital_new['seat'].str.split(" ").str[-1]
    df_recital_new['seat_num'].astype(int)

    
    df_new = pd.concat([df_concert_new, df_chamber_new, df_recital_new])
    result = df_new.drop(['seat'], axis=1)
    return result.sort_index()

In [30]:
# 함수 잘 들어갔는지 test
test_df = pd.read_csv("dataset/2023bigcontest_advanced_sac.csv")
new_test_df = separate_seat(test_df)
new_test_df

,age,gender,membership_type_1,membership_type_2,membership_type_3,membership_type_4,membership_type_5,membership_type_6,tran_date,tran_time,play_date,play_st_time,price,ticket_cancel,discount_type,performance_code,pre_open_date,open_date,genre,place,running_time,intermission,member_yn,seat_floor,seat_block,seat_col,seat_num
0,50.0,F,골드,무료,그린,NaN,NaN,NaN,20220114,1512,20220204,2000,10000,2,일반,302,20220114.0,20220115.0,교향곡,콘서트홀,120,15,Y,3,BOX9,-1,10
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211003,1550,20211009,2000,0,0,초대권,1120,NaN,20210823.0,독주,리사이틀홀,90,15,N,1,-1,7,5
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20220917,1520,20221105,2000,0,0,초대권,2283,20220911.0,20220911.0,독주,IBK챔버홀,80,15,N,1,C,16,3
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190630,1132,20190731,2000,0,0,초대권,2798,NaN,20190701.0,독주,리사이틀홀,90,15,N,1,-1,2,3
4,50.0,M,무료,NaN,NaN,NaN,NaN,NaN,20220206,1615,20220302,1930,180000,0,일반,897,20220109.0,20220110.0,독주,콘서트홀,90,0,Y,1,B,12,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1920863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20190614,920,20190706,1400,0,0,초대권,1265,20190528.0,20190531.0,실내악,IBK챔버홀,100,15,N,1,A,14,6
1920864,20.0,F,무료,싹틔우미,NaN,NaN,NaN,NaN,20221224,1223,20230408,1700,21000,0,청소년(만7세~만24세/본인만)30%,313,NaN,20221223.0,교향곡,콘서트홀,120,0,Y,3,B,5,4
1920865,50.0,F,블루,무료,그린,NaN,NaN,NaN,20210614,129,20210619,2000,24000,0,그린회원 할인20%,736,NaN,20210503.0,독주,리사이틀홀,100,15,Y,1,-1,7,3
1920866,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20211123,1617,20211127,2000,0,0,초대권,1920,20211029.0,20211029.0,독주,IBK챔버홀,90,15,N,1,B,9,5


In [31]:
new_test_df.isnull().sum()

age                  1300183
gender               1299714
membership_type_1    1299714
membership_type_2    1433215
membership_type_3    1749009
membership_type_4    1890675
membership_type_5    1919586
membership_type_6    1920868
tran_date                  0
tran_time                  0
play_date                  0
play_st_time               0
price                      0
ticket_cancel              0
discount_type              0
performance_code           0
pre_open_date         864385
open_date                340
genre                  25842
place                      0
running_time               0
intermission               0
member_yn                  0
seat_floor                 0
seat_block                 0
seat_col                   0
seat_num                   0
dtype: int64

In [48]:
new_test_df[['seat_floor','seat_block','seat_col','seat_num']].sample(10)

,seat_floor,seat_block,seat_col,seat_num
1631516,2,BOX3,1,12
879717,4,G,2,18
212680,3,D,5,5
173441,1,A,14,6
903401,1,A,10,6
1656959,1,A,6,1
1679411,1,C,9,12
209925,2,BOX3,-1,7
1410507,1,-1,5,3
1671143,1,B,18,10


In [ ]:
# 블록은 BOXn, A,B,C, 좌측, 우측 이런 식으로 되어 있음. 임의의 int 값을 부여해야하지 않을지? 추후 논의.

# DISCOUNT 처리 : 예정